# Data Wrangling with PySpark and SQL Spark



In [2]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import ArrayType, BooleanType, LongType, FloatType, IntegerType
from pyspark.sql.functions import lit, udf, struct, countDistinct, collect_list, avg, count, col
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml import Pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [15]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()
user_log.createOrReplaceTempView("log_table")

In [4]:
path = "mini_sparkify_event_data.json"
user_log = spark.read.json(path)

# Data Exploration 

Now we explore the data

In [5]:
user_log.count()

286500

In [8]:
user_log.describe("artist").show()

+-------+------------------+
|summary|            artist|
+-------+------------------+
|  count|            228108|
|   mean| 551.0852017937219|
| stddev|1217.7693079161377|
|    min|               !!!|
|    max| ÃÂlafur Arnalds|
+-------+------------------+



In [9]:
user_log.describe("sessionId").show()

+-------+-----------------+
|summary|        sessionId|
+-------+-----------------+
|  count|           286500|
|   mean|1041.526554973822|
| stddev|726.7762634630807|
|    min|                1|
|    max|             2474|
+-------+-----------------+



In [12]:
user_log.select("page").dropDuplicates().sort("page").show()

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|               Login|
|              Logout|
|            NextSong|
|            Register|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
| Submit Registration|
|      Submit Upgrade|
|         Thumbs Down|
+--------------------+
only showing top 20 rows



In [6]:
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). hour)

In [7]:
user_log = user_log.withColumn("hour", get_hour(user_log.ts))

In [9]:
songs_in_hour = user_log.filter(user_log.page == "NextSong").groupby(user_log.hour).count().orderBy(user_log.hour.cast("float"))

## How many females are in this dataset?

In [12]:
user_log.filter(user_log.gender == 'F') \
    .select('userId', 'gender') \
    .dropDuplicates() \
    .count()

104

In [ ]:
We can also use Spark SQL instead of spark

In [16]:
spark.sql("SELECT COUNT(DISTINCT userID) \
            FROM log_table \
            WHERE gender = 'F'").show()

+----------------------+
|count(DISTINCT userID)|
+----------------------+
|                   104|
+----------------------+



## Who is the most played artist and how many songs did they play?

In [18]:
user_log.filter(user_log.page == 'NextSong') \
    .select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)','Artistcount') \
    .show()

+--------------------+-----------+
|              Artist|Artistcount|
+--------------------+-----------+
|      The Black Keys|       1125|
|         Silverstein|         60|
|           Kate Nash|         99|
|        Yann Tiersen|        139|
|    Jane's Addiction|         68|
|          Tim Hughes|         39|
|          Carl Craig|          3|
|Dashboard Confess...|         98|
|Yonder Mountain S...|         63|
|           Los Lobos|         45|
|Pete Rock & C.L. ...|          2|
|        Ziggy Marley|         28|
|      Jarabe De Palo|         83|
|               Rufio|         13|
|WC And The Maad C...|          1|
|      Jorge Gonzalez|         24|
|                Silk|         22|
|  The Watts Prophets|          6|
|            La Shica|          2|
|        Generation X|         19|
+--------------------+-----------+
only showing top 20 rows



In [17]:
spark.sql("SELECT Artist, COUNT(Artist) AS plays \
        FROM log_table \
        GROUP BY Artist \
        ORDER BY plays DESC \
        LIMIT 1").show()

+-------------+-----+
|       Artist|plays|
+-------------+-----+
|Kings Of Leon| 1841|
+-------------+-----+

